# Anatomy of ring and motor-related neurons in the hindbrain

In [ ]:
%matplotlib widget
# %gui qt
from pathlib import Path
from shutil import copy

import flammkuchen as fl
import napari
import numpy as np
import pandas as pd
from lotr import DATASET_LOCATION, LotrExperiment, dataset_folders
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
new_dataset_folders = [d for d in dataset_folders if not (d / "centroid.txt").exists()]
i = 0
all_centroids = dict()

In [ ]:
path = new_dataset_folders[i]
# for path in dataset_folders[:2]:
exp = LotrExperiment(path)
v = napari.Viewer()
anatomy_stack = exp.anatomy_stack
roi_stack = exp.rois_stack + 1

v.add_image(anatomy_stack, contrast_limits=(0, 30))
v.add_labels(roi_stack)
pts = v.add_points([], ndim=3)

In [ ]:
all_centroids[path.name] = pts.data
i += 1
v.close()

In [ ]:
all_centroids

In [ ]:
fl.save("/Users/luigipetrucco/Desktop/centroids.h5", all_centroids)

In [ ]:
vals.shape

In [ ]:
for path, (name, vals) in zip(new_dataset_folders, all_centroids.items()):
    assert path.name == name

    with open(path / "centroid.txt", "w") as f:
        f.writelines([f"{vals[0, i]}\n" for i in [0, 2, 1]])

    transform_mat = np.eye(4)
    transform_mat[:3, 3] = -vals[:, [0, 2, 1]]

    np.save(path / "centering_mtx.npy", transform_mat)

# Look at results

In [ ]:
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    coords = exp.coords_um
    cent_coords = exp.morphed_coords_um

    data_dict = {f"c{i}": coords[:, i] for i in range(3)}
    data_dict.update({f"centered{i}": cent_coords[:, i] for i in range(3)})
    data_dict["fid"] = path.name
    # data_dict["new"] = path in new_dataset_folders

    data_df.append(pd.DataFrame(data_dict))

data_df = pd.concat(data_df, ignore_index=1)

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(7, 3))
sel = data_df  # [~data_df["new"]]
axs[0].scatter(
    sel["centered1"], sel["centered2"], lw=0, s=5, alpha=0.03, color=(0.3,) * 3,
)
axs[1].scatter(
    sel["centered1"], sel["centered0"] * 10, lw=0, s=5, alpha=0.03, color=(0.3,) * 3,
)
for i in range(2):
    # axs[i].set_title(k)
    # axs[i].axis("equal")
    axs[i].axis("off")

## Create average lighsheet anatomy

In [ ]:
from scipy.ndimage import affine_transform

In [ ]:
def _to_uint255(stack, lower_percent=1, higher_percent=99.99):
    new_stack = stack.copy()
    new_stack -= np.percentile(new_stack, lower_percent)
    new_stack[new_stack < 0] = 0

    new_stack *= 255 / np.percentile(new_stack, higher_percent)
    new_stack[new_stack > 255] = 255

    return new_stack  # .astype(np.uint8)

In [ ]:
final_res_um = 0.5
final_shape_vox = (200, 600, 550)
center_vox = (50, 450, 225)

centering_mat = np.eye(4)
centering_mat[:, 3] = np.concatenate([-np.array(center_vox), [1,]])

In [ ]:
trasformed = np.zeros((len(dataset_folders),) + final_shape_vox, dtype=np.uint8)

for i, path in tqdm(enumerate(dataset_folders)):
    exp = LotrExperiment(path)
    norm_stack = _to_uint255(exp.anatomy_stack)
    res_mat = np.eye(4)
    np.fill_diagonal(res_mat[:3, :3], np.array([final_res_um,] * 3) / exp.voxel_size_um)

    # Invert values thought for point transformation:
    trasf_mat = exp.trasform_mat_to_lsref.copy()
    trasf_mat[1, 3] = exp.trasform_mat_to_lsref[2, 3]
    trasf_mat[2, 3] = exp.trasform_mat_to_lsref[1, 3]

    trasformed_stack = affine_transform(
        norm_stack,
        # np.eye(4),
        np.linalg.inv(trasf_mat) @ res_mat @ centering_mat,
        # trasf_mat,
        output_shape=final_shape_vox,
    )
    # print(trasformed_stack.min(), trasformed_stack.max())
    trasformed[i, :, :, :] = _to_uint255(trasformed_stack)


trasformed = np.array(trasformed)  # .mean(0)

In [ ]:
mean_trasf = np.mean(trasformed, 0)

In [ ]:
v = napari.view_image(mean_trasf)

In [ ]:
v.close()

In [ ]:
np.save(mean_strasf,)

In [ ]:
from tifffile import imsave

In [ ]:
imsave(
    DATASET_LOCATION / "anatomy" / "mean_ls_anatomy.tiff",
    _to_uint255(mean_trasf),
)

### Trasform to ipn reference coordinates

In [ ]:
from bg_atlasapi.core import Atlas
from bg_space import AnatomicalSpace
from scipy.ndimage import affine_transform
from tifffile import imread

mean_trasf = imread(DATASET_LOCATION / "anatomy" / "mean_ls_anatomy.tiff",)
final_res_um = 0.5
final_shape_vox = mean_trasf.shape

ipn_ref = Atlas(DATASET_LOCATION / "anatomy" / "ipn_zfish_0.5um_v1.6")

In [ ]:
ls_functional_ref = AnatomicalSpace(
    "ipl", shape=final_shape_vox, resolution=(final_res_um,) * 3
)

In [ ]:
ipn_ref.space.resolution

In [ ]:
mean_ls_in_ipn = ls_functional_ref.map_stack_to(ipn_ref.space, mean_trasf)

In [ ]:
ls_functional_to_ipnref_coords = ls_functional_ref.transformation_matrix_to(
    ipn_ref.space
)

In [ ]:
v = napari.view_image(ipn_ref.additional_references["gad1b_gal4"], colormap="magenta")
v.add_image(mean_ls_in_ipn, colormap="green", blending="additive")

In [ ]:
v.close()

In [ ]:
ls_functional_to_ipnref = np.eye(4)
ls_functional_to_ipnref[:-1, 3] = [-90, 0, -120]

In [ ]:
v.layers["mean_ls_in_ipn"].affine = ls_functional_to_ipnref

## Test coords trasformation - ls_functional to ipnref

In [ ]:
from lotr.data_preprocessing.anatomy import transform_points

In [ ]:
test_pts = np.array([(50, 450, 225), [0, 0, 0], mean_trasf.shape])

trasformed_pts = transform_points(
    test_pts, ls_functional_to_ipnref @ ls_functional_to_ipnref_coords
)

In [ ]:
v.close()

In [ ]:
v = napari.view_image(ipn_ref.additional_references["gad1b_gal4"], colormap="magenta")
v.add_image(
    mean_ls_in_ipn,
    colormap="green",
    blending="additive",
    affine=ls_functional_to_ipnref,
)
v.add_points(trasformed_pts, n_dimensional=True, size=30)

## Test coords trasformation - exp to ls_functional

In [ ]:
exp = LotrExperiment(dataset_folders[0])
pts = exp.morphed_coords_um

In [ ]:
pts_centering_mat = centering_mat.copy()
pts_centering_mat[:3, 3] = -pts_centering_mat[:3, 3]


pts[:, [1, 2]] = pts[:, [2, 1]]
trasf_pts = transform_points(pts, pts_centering_mat)

In [ ]:
v = napari.view_image(mean_trasf, colormap="magenta")
v.add_points(
    trasf_pts, n_dimensional=True, size=30,
)

In [ ]:
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    coords = exp.coords_um
    cent_coords = exp.morphed_coords_um

    data_dict = {f"c{i}": coords[:, i] for i in range(3)}
    data_dict.update({f"centered{i}": cent_coords[:, i] for i in range(3)})
    data_dict["fid"] = path.name
    data_dict["hdn"] = np.zeros(exp.n_rois)
    data_dict["hdn"][exp.hdn_indexes] = 1
    # data_dict["new"] = path in new_dataset_folders

    data_df.append(pd.DataFrame(data_dict))

data_df = pd.concat(data_df, ignore_index=1)

In [ ]:
pts = data_df[[f"centered{i}" for i in range(3)]].values

In [ ]:
coords_to_ls_functional = np.zeros((4, 4))
coords_to_ls_functional[0, 0] = 1 / final_res_um
coords_to_ls_functional[-1, -1] = 1
coords_to_ls_functional[1, 2] = 1 / final_res_um
coords_to_ls_functional[2, 1] = 1 / final_res_um

In [ ]:
pts_centering_mat = centering_mat.copy()
pts_centering_mat[:3, 3] = -pts_centering_mat[:3, 3]

trasf_pts = transform_points(pts, pts_centering_mat @ coords_to_ls_functional)

In [ ]:
v = napari.view_image(mean_trasf, colormap="magenta")
v.add_points(trasf_pts, n_dimensional=True, size=10)

## Experiment to IPN ref

In [ ]:
final_scaling = np.zeros((4, 4))
np.fill_diagonal(final_scaling, [0.5,] * 3)

centered_to_ipnref = (
    final_scaling
    @ ls_functional_to_ipnref
    @ ls_functional_to_ipnref_coords
    @ pts_centering_mat
    @ coords_to_ls_functional
)

In [ ]:
centered_to_ipnref

In [ ]:
trasf_pts = transform_points(pts, centered_to_ipnref)

In [ ]:
v = napari.view_image(ipn_ref.additional_references["gad1b_gal4"], scale=(0.5,) * 3)
v.add_points(
    trasf_pts,
    n_dimensional=True,
    size=6,
    edge_width=0,
    opacity=0.1,
    blending="additive",
    face_color="blue",
)
v.add_points(
    trasf_pts[data_df["hdn"].astype(bool), :],
    n_dimensional=True,
    size=6,
    edge_width=0,
    opacity=0.1,
    face_color="magenta",
    blending="additive",
)

## Test lightsheet to IPN ref with class method

In [ ]:
exp = LotrExperiment(dataset_folders[0])

In [ ]:
v = napari.view_image(ipn_ref.additional_references["gad1b_gal4"], scale=(0.5,) * 3)
v.add_points(
    exp.ipnref_coords_um,
    n_dimensional=True,
    size=6,
    edge_width=0,
    opacity=0.1,
    blending="additive",
    face_color="blue",
)
v.add_points(
    exp.ipnref_coords_um[exp.hdn_indexes, :],
    n_dimensional=True,
    size=6,
    edge_width=0,
    opacity=0.1,
    face_color="magenta",
    blending="additive",
)

## IPN to MPIN trasformation

In [ ]:
# Getting the transformation values:
xform1_path = fish_path / "anatomy_affine_res_2avg.xform" / "registration"
trans_values1 = get_xform_values(xform1_path)

# Transforming the coordinates: 
t_matrix1 = values2matrix(trans_values1)
center_vals1 = trans_values1[4]
num_coords = np.shape(coords)[0]
t_coords = np.ones_like(coords)
tr_coords = np.ones_like(coords)

# Removing center values: 
for i in range(num_coords):
    t_coords[i,0] = coords[i,1] - center_vals1[0]
    t_coords[i,1] = coords[i,2] - center_vals1[1]
    t_coords[i,2] = coords[i,0] - center_vals1[2]

t_coords = t_matrix1 @ t_coords.T
t_coords = t_coords.T

for i in range(num_coords):
    tr_coords[i,0] = t_coords[i,2] + center_vals1[2]
    tr_coords[i,1] = t_coords[i,0] + center_vals1[0]
    tr_coords[i,2] = t_coords[i,1] + center_vals1[1]